In [33]:
# Imports necesarios
import numpy as np
import pandas as pd
#import tensorflow as tf
#from tensorflow.keras.models import load_model
#from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold, GridSearchCV
from tqdm import tqdm  # Para mostrar el progreso
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cargar el archivo .npz
loaded_data = np.load('/content/drive/MyDrive/DeepLearning/data_arrays.npz')

# Extraer los arrays
X_train_np = loaded_data['X_train']
X_val_np = loaded_data['X_val']
X_test_np = loaded_data['X_test']
y_train_np = loaded_data['y_train']
y_val_np = loaded_data['y_val']
y_test_np = loaded_data['y_test']

In [4]:
# Cargar el archivo .npz
loaded_image_data = np.load('/content/drive/MyDrive/DeepLearning/preprocessed_images.npz')

# Extraer los arrays de imágenes preprocesadas
X_train_preprocessed = loaded_image_data['X_train_preprocessed']
X_val_preprocessed = loaded_image_data['X_val_preprocessed']
X_test_preprocessed = loaded_image_data['X_test_preprocessed']

In [5]:
# Cargar los modelos
model_tabular = load_model('/content/drive/MyDrive/DeepLearning/modelo_tabular.keras')
model_images = load_model('/content/drive/MyDrive/DeepLearning/modelo_resnet50.keras')


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 8 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [6]:
# Asegurarse de que los modelos no sean entrenables
model_tabular.trainable = False
model_images.trainable = False

In [7]:
# Realizar predicciones
predictions_tabular_train = model_tabular.predict(X_train_np)
predictions_tabular_val = model_tabular.predict(X_val_np)
predictions_tabular_test = model_tabular.predict(X_test_np)
predictions_images_train = model_images.predict(X_train_preprocessed)
predictions_images_val = model_images.predict(X_val_preprocessed)
predictions_images_test = model_images.predict(X_test_preprocessed)


59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
59/59 ━━━━━━━━━━━━━━━━━━━━ 360s 6s/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 126s 6s/step
20/20 ━━━━━━━━━━━━━━━━━━━━ 116s 6s/step


# Utilizamos Grid Search con Random Forest

In [32]:
# Combinar las predicciones de los modelos base para el conjunto de entrenamiento
X_meta_train = np.hstack([predictions_tabular_train, predictions_images_train])

# Combinar las predicciones para el conjunto de validación
X_meta_val = np.hstack([predictions_tabular_val, predictions_images_val])

# Combinar las predicciones de los modelos base para el conjunto de prueba
X_meta_test = np.hstack([predictions_tabular_test, predictions_images_test])

# Definir el espacio de hiperparámetros para Grid Search
param_grid = {
    'n_estimators': [50, 100, 200],  # Número de árboles en el bosque
    'max_depth': [None, 10, 20, 30],  # Profundidad máxima del árbol
    'min_samples_split': [2, 5, 10],  # Mínimo número de muestras requeridas para dividir un nodo
    'min_samples_leaf': [1, 2, 4],    # Mínimo número de muestras en una hoja
}

# Crear el meta-modelo (Random Forest)
meta_model = RandomForestClassifier(random_state=42)

# Configurar GridSearchCV
grid_search = GridSearchCV(estimator=meta_model,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                           n_jobs=-1,
                           verbose=2)

# Realizar la búsqueda en cuadrícula
grid_search.fit(X_meta_train, y_train_np)

# Obtener los mejores parámetros y el mejor modelo
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(f"Mejores parámetros: {best_params}")

# Realizar predicciones en el conjunto de validación con el mejor modelo
val_predictions = best_model.predict(X_meta_val)

# Evaluar el rendimiento en el conjunto de validación
val_accuracy = accuracy_score(y_val_np, val_predictions)
print(f"\nPrecisión en el conjunto de validación: {val_accuracy:.4f}")

print("\nInforme de clasificación (Validación):")
print(classification_report(y_val_np, val_predictions))

# Realizar predicciones en el conjunto de prueba con el mejor modelo
final_predictions = best_model.predict(X_meta_test)

# Evaluar el rendimiento en el conjunto de prueba
accuracy_test = accuracy_score(y_test_np, final_predictions)
print(f"\nPrecisión en el conjunto de prueba: {accuracy_test:.4f}")

print("\nInforme de clasificación (Prueba):")
print(classification_report(y_test_np, final_predictions))

# Analizar la importancia de las características (modelos base)
feature_importance = best_model.feature_importances_
num_features_per_model = predictions_tabular_train.shape[1]

print("\nImportancia de las características:")
for i, importance in enumerate(feature_importance):
    model_name = "Tabular" if i < num_features_per_model else "Imágenes"
    feature_index = i % num_features_per_model
    print(f"{model_name} - Característica {feature_index + 1}: {importance:.4f}")

# Calcular la importancia total de cada modelo base
tabular_importance = np.sum(feature_importance[:num_features_per_model])
images_importance = np.sum(feature_importance[num_features_per_model:])

print(f"\nImportancia total del modelo tabular: {tabular_importance:.4f}")
print(f"Importancia total del modelo de imágenes: {images_importance:.4f}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Mejores parámetros: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 100}

Precisión en el conjunto de validación: 0.5942

Informe de clasificación (Validación):
              precision    recall  f1-score   support

           0       0.70      0.64      0.67       197
           1       0.49      0.49      0.49       213
           2       0.61      0.66      0.63       216

    accuracy                           0.59       626
   macro avg       0.60      0.60      0.60       626
weighted avg       0.60      0.59      0.59       626


Precisión en el conjunto de prueba: 0.5901

Informe de clasificación (Prueba):
              precision    recall  f1-score   support

           0       0.69      0.70      0.69       198
           1       0.48      0.49      0.48       211
           2       0.61      0.59      0.60       218

    accuracy                           0.59       627
   macro a

# Utilizamos Validación Cruzada con Random Forest

In [26]:
# Combinar las predicciones de los modelos base para el conjunto de entrenamiento
X_meta_train = np.hstack([predictions_tabular_train, predictions_images_train])

# Combinar las predicciones para el conjunto de validación
X_meta_val = np.hstack([predictions_tabular_val, predictions_images_val])

# Combinar las predicciones de los modelos base para el conjunto de prueba
X_meta_test = np.hstack([predictions_tabular_test, predictions_images_test])

# Crear el meta-modelo (Random Forest)
meta_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Realizar validación cruzada en el conjunto de entrenamiento
cv_scores = cross_val_score(meta_model, X_meta_train, y_train_np, cv=5)
print(f"Puntuaciones de validación cruzada: {cv_scores}")
print(f"Precisión media de validación cruzada: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

# Entrenar el meta-modelo en todo el conjunto de entrenamiento
meta_model.fit(X_meta_train, y_train_np)

# Realizar predicciones en el conjunto de validación
val_predictions = meta_model.predict(X_meta_val)

# Evaluar el rendimiento en el conjunto de validación
val_accuracy = accuracy_score(y_val_np, val_predictions)
print(f"\nPrecisión en el conjunto de validación: {val_accuracy:.4f}")

print("\nInforme de clasificación (Validación):")
print(classification_report(y_val_np, val_predictions))

# Realizar predicciones en el conjunto de prueba
final_predictions = meta_model.predict(X_meta_test)

# Evaluar el rendimiento en el conjunto de prueba
accuracy_test = accuracy_score(y_test_np, final_predictions)
print(f"\nPrecisión en el conjunto de prueba: {accuracy_test:.4f}")

print("\nInforme de clasificación (Prueba):")
print(classification_report(y_test_np, final_predictions))

# Analizar la importancia de las características (modelos base)
feature_importance = meta_model.feature_importances_
num_features_per_model = predictions_tabular_train.shape[1]

print("\nImportancia de las características:")
for i, importance in enumerate(feature_importance):
    model_name = "Tabular" if i < num_features_per_model else "Imágenes"
    feature_index = i % num_features_per_model
    print(f"{model_name} - Característica {feature_index + 1}: {importance:.4f}")

# Calcular la importancia total de cada modelo base
tabular_importance = np.sum(feature_importance[:num_features_per_model])
images_importance = np.sum(feature_importance[num_features_per_model:])

print(f"\nImportancia total del modelo tabular: {tabular_importance:.4f}")
print(f"Importancia total del modelo de imágenes: {images_importance:.4f}")

Puntuaciones de validación cruzada: [0.95744681 0.92819149 0.90691489 0.904      0.90933333]
Precisión media de validación cruzada: 0.9212 (+/- 0.0400)

Precisión en el conjunto de validación: 0.6022

Informe de clasificación (Validación):
              precision    recall  f1-score   support

           0       0.71      0.66      0.69       197
           1       0.50      0.49      0.49       213
           2       0.61      0.66      0.63       216

    accuracy                           0.60       626
   macro avg       0.61      0.60      0.60       626
weighted avg       0.60      0.60      0.60       626


Precisión en el conjunto de prueba: 0.5885

Informe de clasificación (Prueba):
              precision    recall  f1-score   support

           0       0.69      0.70      0.69       198
           1       0.48      0.49      0.48       211
           2       0.61      0.59      0.60       218

    accuracy                           0.59       627
   macro avg       0.59    

# Utilizamos K-Fold Cross-Validation para un Weighted Average Ensemble, con las predicciones de Validación para calcular los pesos de los modelos

In [25]:
# Cargar los datos
X_tabular = X_val_np  # Datos tabulares
X_images = X_val_preprocessed  # Datos de imágenes preprocesadas
y = y_train_np  # Etiquetas

# Configurar K-Fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5 pliegues

accuracies_tabular = []
accuracies_images = []
all_predictions_tabular = []
all_predictions_images = []

# Usar tqdm para mostrar el progreso en el bucle de validación cruzada
for train_index, val_index in tqdm(kf.split(X_tabular), total=kf.get_n_splits(), desc="Validación Cruzada"):
    # Dividir los datos en conjuntos de entrenamiento y validación para este pliegue
    X_tabular_val_fold = X_tabular[val_index]
    X_images_val_fold = X_images[val_index]
    y_val_fold = y[val_index]

    # Realizar predicciones en el conjunto de validación usando los modelos preentrenados
    predictions_tabular = model_tabular.predict(X_tabular_val_fold)
    predictions_images = model_images.predict(X_images_val_fold)

    # Obtener la clase final para cada modelo
    final_predictions_tabular = np.argmax(predictions_tabular, axis=1)
    final_predictions_images = np.argmax(predictions_images, axis=1)

    # Evaluar el rendimiento de cada modelo en este pliegue
    accuracy_tabular = accuracy_score(y_val_fold, final_predictions_tabular)
    accuracy_images = accuracy_score(y_val_fold, final_predictions_images)

    accuracies_tabular.append(accuracy_tabular)
    accuracies_images.append(accuracy_images)

    # Almacenar las predicciones finales para este pliegue
    all_predictions_tabular.append(final_predictions_tabular)
    all_predictions_images.append(final_predictions_images)

# Calcular la precisión media y desviación estándar para cada modelo
mean_accuracy_tabular = np.mean(accuracies_tabular)
std_accuracy_tabular = np.std(accuracies_tabular)

mean_accuracy_images = np.mean(accuracies_images)
std_accuracy_images = np.std(accuracies_images)

print(f"Precisión media (Tabular): {mean_accuracy_tabular:.4f} ± {std_accuracy_tabular:.4f}")
print(f"Precisión media (Imágenes): {mean_accuracy_images:.4f} ± {std_accuracy_images:.4f}")

# Calcular pesos basados en precisiones medias
total_accuracy = mean_accuracy_tabular + mean_accuracy_images

peso_tabular = mean_accuracy_tabular / total_accuracy
peso_imagenes = mean_accuracy_images / total_accuracy

print(f"Peso del modelo tabular: {peso_tabular:.4f} ({peso_tabular * 100:.2f}%)")
print(f"Peso del modelo de imágenes: {peso_imagenes:.4f} ({peso_imagenes * 100:.2f}%)")

# Concatenar todas las predicciones para análisis posterior
y_true_combined = np.concatenate([y[val_index] for _, val_index in kf.split(X_tabular)])

all_predictions_tabular_combined = np.concatenate(all_predictions_tabular)
all_predictions_images_combined = np.concatenate(all_predictions_images)

print("\nInforme de clasificación (Tabular):")
print(classification_report(y_true_combined, all_predictions_tabular_combined))

print("\nInforme de clasificación (Imágenes):")
print(classification_report(y_true_combined, all_predictions_images_combined))

# Realizar predicciones en el conjunto de prueba
predictions_tabular_test = model_tabular.predict(X_test_np)  # Predicciones del modelo tabular
predictions_images_test = model_images.predict(X_test_preprocessed)  # Predicciones del modelo de imágenes

# Obtener la clase final para cada modelo en el conjunto de prueba
final_predictions_tabular_test = np.argmax(predictions_tabular_test, axis=1)
final_predictions_images_test = np.argmax(predictions_images_test, axis=1)

# Combinar las predicciones usando los pesos calculados
predictions_combined_test = (
    (peso_tabular * predictions_tabular_test) +
    (peso_imagenes * predictions_images_test)
)

# Convertir a clases finales (usando argmax o redondeo según sea necesario)
final_predictions_classes_combined = np.argmax(predictions_combined_test, axis=1)

# Evaluar el rendimiento del modelo combinado en el conjunto de prueba
accuracy_combined_test = accuracy_score(y_test_np, final_predictions_classes_combined)
print(f"\nPrecisión del modelo combinado (Test): {accuracy_combined_test:.4f}")

print("\nInforme de clasificación (Modelo Combinado - Test):")
print(classification_report(y_test_np, final_predictions_classes_combined))

Validación Cruzada:   0%|          | 0/5 [00:00<?, ?it/s]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 6s/step


Validación Cruzada:  20%|██        | 1/5 [00:22<01:29, 22.50s/it]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step


Validación Cruzada:  40%|████      | 2/5 [01:03<01:40, 33.60s/it]

...

Validación Cruzada:  80%|████████  | 4/5 [01:47<00:25, 25.77s/it]

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 23s 5s/step


Validación Cruzada: 100%|██████████| 5/5 [02:29<00:00, 29.87s/it]

Precisión media (Tabular): 0.3291 ± 0.0153
Precisión media (Imágenes): 0.3227 ± 0.0596
Peso del modelo tabular: 0.5049 (50.49%)
Peso del modelo de imágenes: 0.4951 (49.51%)

Informe de clasificación (Tabular):
              precision    recall  f1-score   support

           0       0.37      0.35      0.36       220
           1       0.34      0.35      0.34       217
           2       0.28      0.29      0.28       189

    accuracy                           0.33       626
   macro avg       0.33      0.33      0.33       626
weighted avg       0.33      0.33      0.33       626


Informe de clasificación (Imágenes):
              precision    recall  f1-score   support

           0       0.32      0.27      0.30       220
           1       0.35      0.31      0.33       217
           2       0.30      0.39      0.34       189

    accuracy                           0.32       626
   macro avg       0.32      0.33      0.32       626
weighted avg       0.33      0.32      0.32  

20/20 ━━━━━━━━━━━━━━━━━━━━ 116s 6s/step

Precisión del modelo combinado (Test): 0.6683

Informe de clasificación (Modelo Combinado - Test):
              precision    recall  f1-score   support

           0       0.73      0.78      0.76       198
           1       0.56      0.50      0.53       211
           2       0.70      0.73      0.71       218

    accuracy                           0.67       627
   macro avg       0.66      0.67      0.67       627
weighted avg       0.66      0.67      0.66       627

